In [1]:
import numpy as np
import pandas as pd

In [2]:
products = pd.read_csv('../folder/products.csv', usecols = [0,2,3], dtype = {
            'product_id' : np.uint16,
            'aisle_id' : np.uint8,
            'department_id' : np.int8})

orders = pd.read_csv('../folder/orders.csv', dtype = {
            'order_id': np.int32,
            'user_id': np.int32,
            'eval_set': 'category',
            'order_number': np.int8,
            'order_dow': np.int8,
            'order_hour_of_day': np.int8,
            'days_since_prior_order': np.float32})

In [3]:
priors = pd.read_csv('../folder/order_products__prior.csv', dtype = {
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.uint8,
            'reordered': np.int8})

In [4]:
trains = pd.read_csv('../folder/order_products__train.csv', dtype = {
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.uint8,
            'reordered': np.int8})

In [5]:
priors = priors.join(products.set_index('product_id'),on='product_id')

In [6]:
del products

In [7]:
df1 = priors.groupby(by='order_id').product_id.size().astype(np.int8).reset_index()
df1.rename(columns={'product_id' : 'num_items_order'}, inplace=True)            

In [8]:
priors = priors.join(df1.set_index('order_id'), on='order_id')
priors = priors[['order_id','num_items_order','product_id','reordered','add_to_cart_order','aisle_id','department_id']]

In [9]:
order_train = orders[orders.eval_set=='prior']

In [10]:
order_train = order_train.drop('eval_set', axis=1)

In [11]:
df2 = order_train.groupby(by='user_id').order_number.max().reset_index()
df2.rename(columns={'order_number' : 'num_orders_per_user'}, inplace=True)

In [12]:
order_train = order_train.join(df2.set_index('user_id'), on='user_id')

In [13]:
del df2

In [14]:
order_train = order_train[['user_id','num_orders_per_user','order_number','order_id','order_dow','order_hour_of_day','days_since_prior_order']]

In [15]:
prior_train = order_train.join(priors.set_index('order_id'), on='order_id')

In [16]:
prior_train = prior_train[['user_id','num_orders_per_user','order_number','order_id','num_items_order','product_id','aisle_id','department_id','order_dow','order_hour_of_day','days_since_prior_order','add_to_cart_order','reordered']]

In [17]:
del priors
del order_train

In [18]:
df3 = pd.DataFrame(prior_train.groupby(by='user_id').product_id.value_counts().astype(np.int8))
df3.rename(columns={'product_id':'product_till_now'}, inplace=True)

In [19]:
prior_train = prior_train.join(df3, on=['user_id','product_id'])
prior_train.head(20)

,user_id,num_orders_per_user,order_number,order_id,num_items_order,product_id,aisle_id,department_id,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,product_till_now
0,1,10,1,2539329,5,196,77,7,2,8,NaN,1,0,10
0,1,10,1,2539329,5,14084,91,16,2,8,NaN,2,0,1
0,1,10,1,2539329,5,12427,23,19,2,8,NaN,3,0,10
0,1,10,1,2539329,5,26088,23,19,2,8,NaN,4,0,2
0,1,10,1,2539329,5,26405,54,17,2,8,NaN,5,0,2
1,1,10,2,2398795,6,196,77,7,3,7,15.0,1,1,10
1,1,10,2,2398795,6,10258,117,19,3,7,15.0,2,0,9
1,1,10,2,2398795,6,12427,23,19,3,7,15.0,3,1,10
1,1,10,2,2398795,6,13176,24,4,3,7,15.0,4,0,2
1,1,10,2,2398795,6,26088,23,19,3,7,15.0,5,1,2


In [21]:
del df3

NameError: name 'df3' is not defined

In [22]:
prior_train['ratio_buy_item'] = prior_train.product_till_now.astype(np.float32)/prior_train.num_orders_per_user.astype(np.float32)

In [24]:
prior_train = prior_train[['user_id','num_orders_per_user','order_number','order_id','num_items_order','aisle_id','department_id','order_dow','order_hour_of_day','days_since_prior_order','add_to_cart_order','reordered','product_till_now','ratio_buy_item','product_id']]
prior_train.head(20)

,user_id,num_orders_per_user,order_number,order_id,num_items_order,aisle_id,department_id,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,product_till_now,ratio_buy_item,product_id
0,1,10,1,2539329,5,77,7,2,8,NaN,1,0,10,1.0,196
0,1,10,1,2539329,5,91,16,2,8,NaN,2,0,1,0.1,14084
0,1,10,1,2539329,5,23,19,2,8,NaN,3,0,10,1.0,12427
0,1,10,1,2539329,5,23,19,2,8,NaN,4,0,2,0.2,26088
0,1,10,1,2539329,5,54,17,2,8,NaN,5,0,2,0.2,26405
1,1,10,2,2398795,6,77,7,3,7,15.0,1,1,10,1.0,196
1,1,10,2,2398795,6,117,19,3,7,15.0,2,0,9,0.9,10258
1,1,10,2,2398795,6,23,19,3,7,15.0,3,1,10,1.0,12427
1,1,10,2,2398795,6,24,4,3,7,15.0,4,0,2,0.2,13176
1,1,10,2,2398795,6,23,19,3,7,15.0,5,1,2,0.2,26088


In [29]:
#prior_train = prior_train.set_index(['user_id','order_number','order_dow','order_hour_of_day','days_since_prior_order'])


KeyError: 'user_id'

In [30]:
prior_train.head(40)

num_orders_per_user  \
user_id order_number order_dow order_hour_of_day days_since_prior_order                        
1       1            2         8                 NaN                                      10   
                                                 NaN                                      10   
                                                 NaN                                      10   
                                                 NaN                                      10   
                                                 NaN                                      10   
        2            3         7                 15.0                                     10   
                                                 15.0                                     10   
                                                 15.0                                     10   
                                                 15.0                                     10   
                                                 15.0                                     10   
                                                 15.0                                     10   
        3            3         12                21.0                                     10   
                                                 21.0                                     10   
                                                 21.0                                     10   
                                                 21.0                                     10   
                                                 21.0                                     10   
        4            4         7                 29.0                                     10   
                                                 29.0                                     10   
                                                 29.0                                     10   
                                                 29.0                                     10   
                                                 29.0                                     10   
        5            4         15                28.0                                     10   
                                                 28.0                                     10   
                                                 28.0                                     10   
                                                 28.0                                     10   
                                                 28.0                                     10   
                                                 28.0                                     10   
                                                 28.0                                     10   
                                                 28.0                                     10   
        6            2         7                 19.0                                     10   
                                                 19.0                                     10   
                                                 19.0                                     10   
                                                 19.0                                     10   
        7            1         9                 20.0                                     10   
                                                 20.0                                     10   
                                                 20.0                                     10   
                                                 20.0                                     10   
                                                 20.0                                     10   
        8            1         14                14.0                                     10   
                                                 14.0                                     10   

                                        

In [ ]:
# 추가 가능 feature = ea